# Andalucía, ¿cambio estructural o caso aislado?

## 2. Fusión de datos

Una vez se han recabado y limpiado los principales conjuntos de datos en el notebook anterior, es el momento de reunir toda esta información en un único conjunto, y añadir algunos datos de interés. Concretamente, se pretende 

* Concatenar los resultados de las elecciones generales y municipales, añadiendo una columna para diferenciar el tipo de elección.
* Incluir para cada municipio los datos obtenidos en el anterior notebook.

Además de lo anterior, se añadirán las siguientes columnas:

* Una columna para diferenciar municipios costeros y de interior.
* Una columna con información climática del día de las elecciones.
* Una columna con el bloque gobernante de la Nación en el momento de las elecciones.
* Una columna con el bloque gobernante de la Comunidad Autónoma en el momento de las elecciones.
* Una columna que incluya eventualidades que pudieran haber influído de forma notable en las elecciones (Atentados terroristas de gran magnitud, casos de corrupción especialmente mediáticos, intentos de Golpe de Estado, etc...)
* Dos columnas con el número de partidos que se presentan por bloque y, por potencia mediática y representación, son importantes.

### Índice

* [2.1. Librerías y datos](#1)
* [2.2. Creación del dataframe con los resultados electorales](#2)
* [2.3. Adición de datos referentes a la población](#3)
* [2.4. Adición de los datos referentes a renta](#4)
* [2.5. Adición de datos referentes a inmigración](#5)
* [2.6. Adición de datos referentes a la delincuencia](#6)
* [2.7. Adición de datos referentes a la educación](#7)
* [2.8. Adición de datos referentes al paro](#8)
* [2.9. Otros datos](#9)
    * [2.9.1. Distinción entre municipios costeros y de interior](#9.1)
    * [2.9.2. Gobiernos Nacionales y Autonómicos](#9.2)
    * [2.9.3. Número de partidos importantes por bloque](#9.3)
    * [2.9.4. Eventualidades](#9.4)
    * [2.9.5. Meteorología](#9.5)

### 2.1. Librerías y datos <a class="anchor" id="1"></a>

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
Generales = pd.read_csv('Resultados/Elecciones_Generales.csv')
Autonomicas = pd.read_csv('Resultados/Elecciones_Autonomicas.csv')
Poblacion = pd.read_csv('Resultados/Pob_municipios.csv')
Paro = pd.read_csv('Resultados/Paro_municipios.csv')
Renta = pd.read_csv('Resultados/Renta.csv')
Inmigracion = pd.read_csv('Resultados/Inmigracion.csv')
Delincuencia = pd.read_csv('Resultados/Delincuencia.csv')
Educacion = pd.read_csv('Resultados/Educacion.csv')

### 2.2. Creación del dataframe con resultados electorales <a class="anchor" id="2"></a>

En este apartado se creará el dataframe inicial, formado por la unión de los resultados electorales a nivel municipal de elecciones autonómicas y generales a nivel municipal.

In [4]:
Generales.head()

,Provincia,Municipios,Población,Resultado Izquierda,Resultado Derecha,Diferencia,Fecha,Mes,Año
0,Almería,Abla,1755,9.80,90.20,-80.40,197706,6,1977
1,Almería,Abrucena,1828,23.78,76.11,-52.33,197706,6,1977
2,Almería,Adra,15602,39.58,60.26,-20.68,197706,6,1977
3,Almería,Albánchez,1058,20.65,78.84,-58.19,197706,6,1977
4,Almería,Alboloduy,1082,15.98,84.02,-68.04,197706,6,1977


In [5]:
Autonomicas.head()

,Municipios,Año,Resultado Izquierda,Resultado Derecha,Diferencia
0,Abla,1982,33.45,66.54,-33.09
1,Abrucena,1982,63.50,36.49,27.01
2,Adra,1982,58.08,41.92,16.16
3,Albánchez,1982,38.40,61.61,-23.21
4,Alboloduy,1982,37.11,62.89,-25.78


En primer lugar, se añadirá al dataframe de las elecciones autonómicas una columna con la provincia a la que pertenece cada municipio y el mes en el que se produjo la elección.

In [6]:
Autonomicas['Provincia'] = 'Almería'
for i in Autonomicas['Municipios'].unique():
    
    Autonomicas['Provincia'][Autonomicas['Municipios'] == i] = Generales[Generales['Municipios'] == i]['Provincia'].unique()[0]

In [7]:
Autonomicas['Mes'] = 5 
meses = [6,6,6,3,3,3,3,3,12]
for i in range(1,len(Autonomicas['Año'].unique())):
    
    Autonomicas['Mes'][Autonomicas['Año'] == Autonomicas['Año'].unique()[i]] = meses[i-1]

In [8]:
Autonomicas

,Municipios,Año,Resultado Izquierda,Resultado Derecha,Diferencia,Provincia,Mes
0,Abla,1982,33.45,66.54,-33.09,Almería,5
1,Abrucena,1982,63.50,36.49,27.01,Almería,5
2,Adra,1982,58.08,41.92,16.16,Almería,5
3,Albánchez,1982,38.40,61.61,-23.21,Almería,5
4,Alboloduy,1982,37.11,62.89,-25.78,Almería,5
...,...,...,...,...,...,...,...
6708,Villanueva del Ariscal,2018,49.12,50.62,-1.50,Sevilla,12
6709,Villanueva del Río y Minas,2018,64.61,35.12,29.49,Sevilla,12
6710,Villanueva de San Juan,2018,66.60,33.38,33.22,Sevilla,12
6711,Villaverde del Río,2018,56.20,43.75,12.45,Sevilla,12


Se añade el tipo de elección.

In [9]:
Generales['Tipo de Elección'] = 'General'
Autonomicas['Tipo de Elección'] = 'Autonómica'

In [10]:
data = pd.concat([Generales.drop('Fecha', axis = 1),Autonomicas]).sort_values(by = ['Año','Provincia']).reset_index(drop = True)

In [11]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección
0,1977,-80.40,6,Abla,1755.0,Almería,90.20,9.80,General
1,1977,-52.33,6,Abrucena,1828.0,Almería,76.11,23.78,General
2,1977,-20.68,6,Adra,15602.0,Almería,60.26,39.58,General
3,1977,-58.19,6,Albánchez,1058.0,Almería,78.84,20.65,General
4,1977,-68.04,6,Alboloduy,1082.0,Almería,84.02,15.98,General
...,...,...,...,...,...,...,...,...,...
18184,2019,18.04,11,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General
18185,2019,13.84,11,El Viso del Alcor,19191.0,Sevilla,41.89,55.73,General
18186,2019,65.48,11,Cañada Rosal,3307.0,Sevilla,16.04,81.52,General
18187,2019,26.25,11,Isla Mayor,5857.0,Sevilla,35.88,62.13,General


### 2.3. Adición de datos referentes a población <a class="anchor" id="3"></a>

En este apartado se añaden datos referentes a la población de los municipios.

In [12]:
Poblacion

,Municipios,Año,Total,Provincia,Clasificación
0,Abla,2019,1249.0,Almería,2
1,Abla,2018,1235.0,Almería,2
2,Abla,2017,1267.0,Almería,2
3,Abla,2016,1294.0,Almería,2
4,Abla,2015,1342.0,Almería,2
...,...,...,...,...,...
16935,El Viso del Alcor,2002,16355.0,Sevilla,4
16936,El Viso del Alcor,2001,16276.0,Sevilla,4
16937,El Viso del Alcor,2000,16170.0,Sevilla,4
16938,El Viso del Alcor,1999,16036.0,Sevilla,4


Los datos obtenidos en el censo complementarán a los datos de población dados por el Ministerio del Interior para las elecciones generales. En primer lugar se considerará que la población de elecciones concurrentes el mismo año es la misma, de modo que podrán imputarse los valores de población de esos mismos municipios para elecciones generales.

In [13]:
for i in range(len(data)):
    if math.isnan(data['Población'][i]) == True:
        if data['Año'][i] in data[data['Tipo de Elección'] == 'General']['Año'].unique():
            
            try:
                
                data.at[i,'Población'] = int(data[(data['Municipios'] == data['Municipios'][i])&(data['Año'] == data['Año'][i])\
                                                  &(data['Tipo de Elección'] == 'General')]['Población'])
                
            except:
                
                print(data['Año'][i],data['Municipios'][i])

1982 Cóbdar
1982 Trebujena
1982 La Carlota
1982 Dos Torres
1982 La Taha
1982 San Juan del Puerto
1982 Villanueva de las Cruces
1982 Alozaina
1982 Alpandeire
1982 Genalguacil
1982 Rincón de la Victoria
1982 Riogordo
1982 Sierra de Yeguas
1982 Teba
1986 Benaocaz
1986 Las Navas de la Concepción
1986 Los Palacios y Villafranca
1996 Las Navas de la Concepción
1996 Los Palacios y Villafranca


Las excepciones anteriores se deben a que los datos de sus respectivos municipios no aparecen en las tablas de resultados publicadas por el Ministerio del Interior para elecciones generales. El siguiente paso es añadir la población para las elecciones autonómicas que no coincidieron elecciones generales, en cuyo caso

In [14]:
for i in range(len(data)):
    if math.isnan(data['Población'][i]) == True:
        if data['Año'][i] in Poblacion['Año'].unique():
            if data['Municipios'][i] in Poblacion['Municipios'].unique():
            
                data.at[i,'Población'] = Poblacion[(Poblacion['Municipios'] == data['Municipios'][i])&(Poblacion['Año'] ==\
                                                                                                data['Año'][i])]['Total']

In [15]:
len(data[data['Población'].notna()])

16843

Así, obtenemos finalmente que un total de 16 843 municipios de un total de 18189, un 92.6 %, tienen asignado un valor para la población. Ahora, se clasificará a los municipios en función de su población, tal como se hizo durante la recolección de datos pero de forma generalizada y dedicada a las elecciones.

In [16]:
data['Clasificación'] = 1
for i in range(len(data)):
    if data['Población'][i] >= 1000 and data['Población'][i] < 2000:
        data.at[i,'Clasificación'] = 2
    elif data['Población'][i] >= 2000 and data['Población'][i] < 10000:
        data.at[i,'Clasificación'] = 3
    elif data['Población'][i] >= 10000 and data['Población'][i] < 100000:
        data.at[i,'Clasificación'] = 4
    elif data['Población'][i] >= 100000:
        data.at[i,'Clasificación'] = 5

Más adelante se analizará la evolución de la distribución de población elección a elección.

EL último paso en cuestión de población es corregir los errores concretos en los datos de elecciones generales del Ministerio del Interior en los que aparecen dos municipios con valor de población cero. Esto es obviamente imposible, de modo que es necesario cambiarlo. Los municipios se ven a continuación.

In [17]:
data[data['Población'] == 0]

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación
2408,1982,14.40,10,Villarrodrigo,0.0,Jaén,32.14,46.54,General,1
2503,1982,-57.48,5,Villarrodrigo,0.0,Jaén,78.74,21.26,Autonómica,1
2698,1982,37.28,10,Albaida del Aljarafe,0.0,Sevilla,14.04,51.32,General,1
2800,1982,-2.75,5,Albaida del Aljarafe,0.0,Sevilla,50.94,48.19,Autonómica,1


In [18]:
data['Población'][(data['Población'] == 0)&(data['Municipios'] == 'Villarrodrigo')] = 911
data['Población'][(data['Población'] == 0)&(data['Municipios'] == 'Albaida del Aljarafe')] = 1591

### 2.4. Adición de los datos referentes a renta <a class="anchor" id="4"></a>

En este apartado de añadirán al conjunto los datos obtenidos para la renta de los municipios.

In [19]:
Renta

,Municipios,Puesto Nacional,Puesto Autonómico,Renta Bruta,Renta Disponible,Provincia,Año
0,Almería,689,39,22868.0,18837,Almería,2013
1,Carboneras,1067,66,20672.0,17294,Almería,2013
2,Roquetas de Mar,1085,67,20582.0,17072,Almería,2013
3,Huércal de Almería,1124,71,20386.0,17150,Almería,2013
4,Vera,1167,73,20202.0,16861,Almería,2013
...,...,...,...,...,...,...,...
3425,Badolatosa,2817,521,14459.0,13079,Sevilla,2018
3426,Los Corrales,2818,522,14444.0,12969,Sevilla,2018
3427,Villanueva de San Juan,2834,531,14193.0,12908,Sevilla,2018
3428,Marinaleda,2863,550,13739.0,12438,Sevilla,2018


Primero se excluyen los datos de Agrupación de pequeños municipios.

In [20]:
Renta1 = Renta[Renta['Municipios'] != 'Agrupación municipios pequeños'].reset_index(drop = True)

In [21]:
data = data.merge(Renta1, how = 'outer')
data = data[data['Diferencia'].notna()]

In [22]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,Puesto Nacional,Puesto Autonómico,Renta Bruta,Renta Disponible
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,NaN,NaN,NaN,NaN
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,NaN,NaN,NaN,NaN
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,NaN,NaN,NaN,NaN
3,1977,-58.19,6.0,Albánchez,1058.0,Almería,78.84,20.65,General,2.0,NaN,NaN,NaN,NaN
4,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,65.48,11.0,Cañada Rosal,3307.0,Sevilla,16.04,81.52,General,3.0,NaN,NaN,NaN,NaN
18185,2019,30.66,4.0,Isla Mayor,5857.0,Sevilla,33.80,64.46,General,3.0,NaN,NaN,NaN,NaN
18186,2019,26.25,11.0,Isla Mayor,5857.0,Sevilla,35.88,62.13,General,3.0,NaN,NaN,NaN,NaN
18187,2019,12.09,4.0,El Cuervo de Sevilla,8628.0,Sevilla,43.12,55.21,General,3.0,NaN,NaN,NaN,NaN


In [23]:
len(data[data['Renta Bruta'].notna()])

2249

Como se ve, para el conjunto de las elecciones, hay un total de 2249 filas a las que se le asignan valores de renta. Para los municipios pequeños se tomará la media dada por el dato *Agrupación municipios pequeños*, y con él se considerará a todos los municipios incluídos es esa denominación. Idealmente, el número de filas con valores de renta no nulos debe alcanzar los 3080.

In [24]:
Agrup_municipios = Renta[Renta['Municipios'] == 'Agrupación municipios pequeños']

In [25]:
for i in [2015, 2016, 2018]: #Años de elecciones que coinciden con datos de renta.
    a = data[data['Año'] == i]
    for j in a.index:
        if math.isnan(data['Renta Bruta'][j]) == True:
            
            data.at[j,'Renta Bruta'] = Agrup_municipios[(Agrup_municipios['Provincia'] == a['Provincia'][j])&\
                                                        (Agrup_municipios['Año'] == i)]['Renta Bruta']
            data.at[j,'Renta Disponible'] = Agrup_municipios[(Agrup_municipios['Provincia'] == a['Provincia'][j])&\
                                                        (Agrup_municipios['Año'] == i)]['Renta Disponible']

In [26]:
len(data[data['Renta Bruta'].notna()])

3076

De modo que se consigue, aproximadamente, el resultado deseado.

### 2.5. Adición de datos referentes a inmigración <a class="anchor" id="5"></a>

En este apartado se añadirán al conjunto los datos obtenidos para la inmigración por municipios.

In [27]:
Inmigracion

,Municipios,Origen,Total,Año,Provincia
0,Abla,Africa,0.065963,1998,Almería
1,Abla,América,0.000000,1998,Almería
2,Abla,Asia,0.000000,1998,Almería
3,Abla,Europa,0.000000,1998,Almería
4,Abla,Oceanía,0.000000,1998,Almería
...,...,...,...,...,...
84555,El Viso del Alcor,Europa,1.058860,2019,Sevilla
84556,El Viso del Alcor,Africa,1.479290,2019,Sevilla
84557,El Viso del Alcor,América,0.820098,2019,Sevilla
84558,El Viso del Alcor,Asia,0.098619,2019,Sevilla


Lo primero será separar los datos en distintos conjuntos según origen de la inmigración.

In [28]:
Africa = Inmigracion[Inmigracion['Origen'] == 'Africa'].drop('Origen', axis = 1).reset_index(drop = True)
América = Inmigracion[Inmigracion['Origen'] == 'América'].drop('Origen', axis = 1).reset_index(drop = True)
Asia = Inmigracion[Inmigracion['Origen'] == 'Asia'].drop('Origen', axis = 1).reset_index(drop = True)
Europa = Inmigracion[Inmigracion['Origen'] == 'Europa'].drop('Origen', axis = 1).reset_index(drop = True)
Oceanía = Inmigracion[Inmigracion['Origen'] == 'Oceanía'].drop('Origen', axis = 1).reset_index(drop = True)

In [29]:
Africa.rename(columns={'Total':'Inmigración Africa'}, inplace = True)
América.rename(columns={'Total':'Imigración América'}, inplace = True)
Asia.rename(columns={'Total':'Inmigración Asia'}, inplace = True)
Europa.rename(columns={'Total':'Inmigración Europa'}, inplace = True)
Oceanía.rename(columns={'Total':'Inmigración Oceanía'}, inplace = True)

Se compilan los conjuntos en una lista y se unen.

In [30]:
Lista = [Africa, América, Asia, Europa, Oceanía]

for i in Lista:
    
    data = data.merge(i, how = 'outer')

In [31]:
data = data[data['Diferencia'].notna()].sort_values(by = ['Año','Provincia','Municipios']).reset_index(drop = True)

In [32]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,Puesto Nacional,Puesto Autonómico,Renta Bruta,Renta Disponible,Inmigración Africa,Imigración América,Inmigración Asia,Inmigración Europa,Inmigración Oceanía
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1977,-58.74,6.0,Albox,10407.0,Almería,79.30,20.56,General,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,33.03,11.0,Villanueva del Río y Minas,4860.0,Sevilla,31.83,64.86,General,3.0,NaN,NaN,NaN,NaN,0.226431,0.391107,0.102923,1.832030,0.020585
18185,2019,19.71,4.0,Villaverde del Río,7797.0,Sevilla,39.64,59.35,General,3.0,NaN,NaN,NaN,NaN,0.575595,0.550013,0.076746,2.750064,0.000000
18186,2019,18.04,11.0,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General,3.0,NaN,NaN,NaN,NaN,0.575595,0.550013,0.076746,2.750064,0.000000
18187,2019,-5.37,4.0,Écija,39882.0,Sevilla,52.02,46.65,General,4.0,NaN,NaN,NaN,NaN,0.203145,1.121059,0.152986,1.289093,0.005016


### 2.6. Adición de datos referentes a la delincuencia <a class="anchor" id="6"></a>

En este apartado se añadirán los datos de delincuencia por provincia.

In [33]:
Delincuencia

,Provincia,Nacionalidad,Año,Total
0,Almería,Total,2019,6812.0
1,Almería,Total,2018,7234.0
2,Almería,Total,2017,6900.0
3,Almería,Total,2016,6202.0
4,Almería,Total,2015,4944.0
...,...,...,...,...
443,Sevilla,Oceanía,2017,0.0
444,Sevilla,Oceanía,2016,0.0
445,Sevilla,Oceanía,2015,0.0
446,Sevilla,Oceanía,2014,1.0


Como para el caso de la inmigración, se crearán distintos dataframes dependiendo del origen de la misma.

In [34]:
España = Delincuencia[Delincuencia['Nacionalidad'] == 'España'].drop('Nacionalidad', axis = 1).reset_index(drop = True)
Africa = Delincuencia[Delincuencia['Nacionalidad'] == 'África'].drop('Nacionalidad', axis = 1).reset_index(drop = True)
América = Delincuencia[Delincuencia['Nacionalidad'] == 'América'].drop('Nacionalidad', axis = 1).reset_index(drop = True)
Asia = Delincuencia[Delincuencia['Nacionalidad'] == 'Asia'].drop('Nacionalidad', axis = 1).reset_index(drop = True)
Europa = Delincuencia[Delincuencia['Nacionalidad'] == 'Resto de Europa'].drop('Nacionalidad', axis = 1).reset_index(drop = True)
Unión = Delincuencia[Delincuencia['Nacionalidad'] == 'Unión Europea'].drop('Nacionalidad', axis = 1).reset_index(drop = True)
Oceanía = Delincuencia[Delincuencia['Nacionalidad'] == 'Oceanía'].drop('Nacionalidad', axis = 1).reset_index(drop = True)

In [35]:
España.rename(columns={'Total':'Delincuencia España'}, inplace = True)
Africa.rename(columns={'Total':'Delincuencia Africa'}, inplace = True)
América.rename(columns={'Total':'Delincuencia América'}, inplace = True)
Asia.rename(columns={'Total':'Delincuencia Asia'}, inplace = True)
Europa.rename(columns={'Total':'Delincuencia Resto de Europa'}, inplace = True)
Unión.rename(columns={'Total':'Delincuencia Unión Europea'}, inplace = True)
Oceanía.rename(columns={'Total':'Delincuencia Oceanía'}, inplace = True)

Se compilan todos los continentes en una lista y se unen.

In [36]:
Lista = [España, Africa, América, Asia, Unión, Europa, Oceanía]

for i in Lista:
    
    data = data.merge(i, how = 'outer')

In [37]:
data = data[data['Diferencia'].notna()].sort_values(by = ['Año','Provincia','Municipios']).reset_index(drop = True)

In [38]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Inmigración Asia,Inmigración Europa,Inmigración Oceanía,Delincuencia España,Delincuencia Africa,Delincuencia América,Delincuencia Asia,Delincuencia Unión Europea,Delincuencia Resto de Europa,Delincuencia Oceanía
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1977,-58.74,6.0,Albox,10407.0,Almería,79.30,20.56,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,33.03,11.0,Villanueva del Río y Minas,4860.0,Sevilla,31.83,64.86,General,3.0,...,0.102923,1.832030,0.020585,17002.0,358.0,393.0,84.0,531.0,65.0,0.0
18185,2019,19.71,4.0,Villaverde del Río,7797.0,Sevilla,39.64,59.35,General,3.0,...,0.076746,2.750064,0.000000,17002.0,358.0,393.0,84.0,531.0,65.0,0.0
18186,2019,18.04,11.0,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General,3.0,...,0.076746,2.750064,0.000000,17002.0,358.0,393.0,84.0,531.0,65.0,0.0
18187,2019,-5.37,4.0,Écija,39882.0,Sevilla,52.02,46.65,General,4.0,...,0.152986,1.289093,0.005016,17002.0,358.0,393.0,84.0,531.0,65.0,0.0


### 2.7. Adición de datos referentes a la educación <a class="anchor" id="7"></a>

En este apartado se añadirán los datos de nivel educativo para toda la Comunidad.

In [39]:
Educacion.head()

,Formación,Año,Total
0,Analfabeto,2019,2.5
1,Analfabeto,2018,2.6
2,Analfabeto,2017,3.0
3,Analfabeto,2016,3.1
4,Analfabeto,2015,3.4


In [40]:
Educacion['Formación'].unique()

array(['Analfabeto', 'Primaria incompleta', 'Primaria', 'ESO',
       'Bachillerato', 'FP', 'Superior'], dtype=object)

In [41]:
for i in Educacion['Formación'].unique():
    df = Educacion[Educacion['Formación'] == i].drop('Formación',axis = 1)
    df.rename(columns = {'Total': i}, inplace = True)
    data = data.merge(df, how = 'outer')

In [42]:
data = data[data['Diferencia'].notna()]

In [43]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Delincuencia Unión Europea,Delincuencia Resto de Europa,Delincuencia Oceanía,Analfabeto,Primaria incompleta,Primaria,ESO,Bachillerato,FP,Superior
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1977,-58.74,6.0,Albox,10407.0,Almería,79.30,20.56,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,33.03,11.0,Villanueva del Río y Minas,4860.0,Sevilla,31.83,64.86,General,3.0,...,531.0,65.0,0.0,2.5,8.2,12.7,31.2,12.4,8.1,24.9
18185,2019,19.71,4.0,Villaverde del Río,7797.0,Sevilla,39.64,59.35,General,3.0,...,531.0,65.0,0.0,2.5,8.2,12.7,31.2,12.4,8.1,24.9
18186,2019,18.04,11.0,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General,3.0,...,531.0,65.0,0.0,2.5,8.2,12.7,31.2,12.4,8.1,24.9
18187,2019,-5.37,4.0,Écija,39882.0,Sevilla,52.02,46.65,General,4.0,...,531.0,65.0,0.0,2.5,8.2,12.7,31.2,12.4,8.1,24.9


### 2.8. Adición de datos referentes al Paro <a class="anchor" id="8"></a>

Los datos de paro se han dejado para el final por la dificultad de condensar todos los datos de que se disponen. Es un hecho que el desempleo no se sufre únicamente durante la jornada electoral. Por tanto, no tiene sentido considerar únicamente el desempleo en el momento de las elecciones, que en muchas ocasiones no es representativo de los números y tendencias dados a lo largo de una legislatura.

In [44]:
Paro

,Municipios,Total,Agricultura,Industria,Construcción,Servicios,Provincia,Mes,Año
0,Abla,2.923588,0.465116,0.000000,0.465116,1.993355,Almería,enero,2006
1,Abrucena,2.539208,1.418969,0.149365,0.074683,0.896191,Almería,enero,2006
2,Adra,2.662986,0.522404,0.055213,1.388830,0.696539,Almería,enero,2006
3,Albánchez,0.606061,0.000000,0.151515,0.303030,0.151515,Almería,enero,2006
4,Alboloduy,2.063274,0.412655,0.000000,0.275103,1.375516,Almería,enero,2006
...,...,...,...,...,...,...,...,...,...
129355,Villanueva del Ariscal,11.013616,0.529501,0.408472,1.331316,7.806354,Sevilla,diciembre,2019
129356,Villanueva del Río y Minas,10.312886,2.243722,0.802800,1.276245,5.125566,Sevilla,diciembre,2019
129357,Villanueva de San Juan,7.053571,1.339286,0.267857,1.607143,3.392857,Sevilla,diciembre,2019
129358,Villaverde del Río,8.147864,1.202354,0.396521,1.790739,4.054745,Sevilla,diciembre,2019


Por tanto, para cada sector se incluirá una columna con el porcentaje inicial, otra con el final, y otras cuatro con las medias del incremento mes a mes de la legislatura dividida en cuatro partes. Se empezará por las columnas correspondientes al paro inicial de la legislatura anterior a cada elección. Para ello se creará una función que asocie cada valor a la fila y columna correspondiente.

In [45]:
def Paro_inicial(data,Paro,a,b,c, tipo = 'General'):
    
    '''
    Función para calcular el paro inicial en una legislatura por municipio y sector.
    
        Parámetros:
            data (pandas.DataFrame): Dataframe con los datos generales de las elecciones.
            Paro (pandas.DataFrame): Dataframe con los datos de Paro por municipio.
            a (str): Lista de columnas asociadas al Paro inicial.
            b (str): Lista de Años para los que se tienen datos de Paro.
            c (str): Lista de sectores.
            tipo (str): Tipo de elección.
            
        Return:
            data (pandas.DataFrame): El mismo DataFrame de la entrada pero con el Paro inicial calculado.
    '''

    for i in range(len(data)):
        if str(data['Año'][i]) in b:
            for j in range(len(c)):
                if data['Año'][i] == 2008:
            
                    data.at[i,a[j]] = Paro[(Paro['Año'] == 2006)&(Paro['Municipios'] == data['Municipios'][i])&\
                                       (Paro['Mes'] == 'enero')][c[j]]
                    
                elif data['Tipo de Elección'][i] == tipo:
                    try:
                    
                        data.at[i,a[j]] = float(Paro[(Paro['Año'] == int(b[b.index(str(data['Año'][i]))-1]))&\
                                       (Paro['Municipios'] == data['Municipios'][i])&\
                                       (Paro['Mes'] == Paro['Mes'].unique()[int(data[(data['Año'] == \
                                            int(b[b.index(str(data['Año'][i]))-1]))&(data['Municipios'] == \
                                            data['Municipios'][i])&(data['Tipo de Elección'] == tipo)]\
                                                                                ['Mes'])-1])][c[j]])
                    except:
                        pass
                        
    return data

In [46]:
%%time

a = ['Paro Total Inicial','Paro Agricultura Inicial','Paro Industria Inicial','Paro Construcción Inicial',
     'Paro Servicios Inicial']
for i in a:
    data[i] = np.nan

b = ['2008', '2011', '2015','2016', '2019']
c = ['Total','Agricultura','Industria','Construcción','Servicios']

data = Paro_inicial(data,Paro,a,b,c)

b = ['2008','2012','2015','2018']

data = Paro_inicial(data,Paro,a,b,c, tipo = 'Autonómica')

Wall time: 14min 32s


Ahora se hará lo propio con las columnas para el paro final por sector.

In [47]:
def Paro_final(data,Paro,a,b,c):
    
    '''
    Función para calcular el Paro final de una legislatura por municipio y sector.
    
        Parámetros:
            data (pandas.DataFrame): Dataframe con los datos de las elecciones.
            Paro (pandas.DataFrame): Dataframe con los datos de Paro por municipios.
            a (str): Lista de columnas asociadas al Paro inicial.
            b (str): Lista de Años para los que se tienen datos de Paro.
            c (str): Lista de sectores.
            
        Return:
            data (pandas.DataFrame): El mismo Dataframe de entrada con los datos de Paro final calculados.
    '''
    
    for i in range(len(data)):
        if str(data['Año'][i]) in b:
            for j in range(len(c)):
                
                data.at[i,a[j]] = float(Paro[(Paro['Municipios'] == data['Municipios'][i])&\
                                             (Paro['Año'] == int(b[b.index(str(data['Año'][i]))]))&\
                                             (Paro['Mes'] == Paro['Mes'].unique()[int(data['Mes'][i])-1])][c[j]])
                
    return data

In [48]:
%%time

a = ['Paro Total Final','Paro Agricultura Final','Paro Industria Final','Paro Construcción Final','Paro Servicios Final']
for i in a:
    data[i] = np.nan
    
b = ['2008', '2011', '2012', '2015','2016', '2018', '2019']
c = ['Total','Agricultura','Industria','Construcción','Servicios']

data = Paro_final(data,Paro,a,b,c)

Wall time: 11min 59s


In [49]:
data[(data['Paro Total Final'].notna()) & (data['Municipios'] == 'Sevilla')]

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Paro Total Inicial,Paro Agricultura Inicial,Paro Industria Inicial,Paro Construcción Inicial,Paro Servicios Inicial,Paro Total Final,Paro Agricultura Final,Paro Industria Final,Paro Construcción Final,Paro Servicios Final
12019,2008,21.75,3.0,Sevilla,699145.0,Sevilla,38.41,60.16,General,5.0,...,4.463000,0.242187,0.190229,0.574236,3.456348,4.243747,0.210072,0.165771,0.499029,3.368874
12020,2008,20.22,3.0,Sevilla,699145.0,Sevilla,39.89,60.11,Autonómica,5.0,...,4.463000,0.242187,0.190229,0.574236,3.456348,4.243747,0.210072,0.165771,0.499029,3.368874
12800,2011,9.46,11.0,Sevilla,704198.0,Sevilla,44.41,53.87,General,5.0,...,4.243747,0.210072,0.165771,0.499029,3.368874,3.469171,0.177377,0.125601,0.371682,2.794511
13561,2012,12.87,3.0,Sevilla,702355.0,Sevilla,43.43,56.30,Autonómica,5.0,...,4.243747,0.210072,0.165771,0.499029,3.368874,3.228140,0.164162,0.118174,0.319497,2.626307
15087,2015,10.06,12.0,Sevilla,696676.0,Sevilla,44.60,54.66,General,5.0,...,3.469171,0.177377,0.125601,0.371682,2.794511,11.796166,0.205656,0.695511,1.041826,8.158062
15088,2015,18.34,3.0,Sevilla,696676.0,Sevilla,40.77,59.11,Autonómica,5.0,...,3.228140,0.164162,0.118174,0.319497,2.626307,12.535201,0.224391,0.770452,1.155390,8.571247
15869,2016,2.96,6.0,Sevilla,693878.0,Sevilla,48.09,51.05,General,5.0,...,11.796166,0.205656,0.695511,1.041826,8.158062,11.688383,0.211855,0.660183,0.996284,8.198058
16637,2018,-2.05,12.0,Sevilla,688711.0,Sevilla,50.91,48.86,Autonómica,5.0,...,12.535201,0.224391,0.770452,1.155390,8.571247,9.936534,0.203423,0.532299,0.734706,7.140586
18165,2019,3.72,4.0,Sevilla,688711.0,Sevilla,47.51,51.23,General,5.0,...,11.688383,0.211855,0.660183,0.996284,8.198058,9.946383,0.212608,0.519756,0.706659,7.136592
18166,2019,3.33,11.0,Sevilla,688711.0,Sevilla,47.52,50.85,General,5.0,...,11.688383,0.211855,0.660183,0.996284,8.198058,9.877547,0.214205,0.489985,0.694315,7.117132


Por último, queda la cuestión más complicada: la construcción de columnas con las medias del incremento a lo largo de cada legislatura dividido en cuatro. El principal problema radica en que no pueden organizarse columnas con los datos en bruto debido a la duración dispar de las legislaturas. Por tanto, dado que se entiende que es importante el incremento de la tasa de paro más que al propio número, es razonable que sea esta medida la que quede plasmada en el dataset final.

In [50]:
def Incrementos(data, Paro, c, tipo = 'General'):
    
    '''
    Función que calcula los incrementos de los datos del paro a lo largo de las legislaturas divididas en cuatro partes por
    sector.
    
        Parámetros:
            data (pandas.DataFrame): Dataframe con los datos generales de las elecciones.
            Paro (pandas.DataFrame): Dataframe con los datos de Paro por municipios.
            c (str): Lista de sectores de los que calcular el incremento.
            tipo (str): Tipo de elección.
            
        Return:
            data(pandas.DataFrame): El mismo Dataframe de la entrada pero con los incrementos calculados.
    '''
    
    A = data[data['Tipo de Elección'] == tipo]
    for i in A['Municipios'].unique():
        B = A[A['Municipios'] == i]
        C = Paro[Paro['Municipios'] == i].reset_index(drop = True)
        
        for j in range(len(B['Año'].unique())+1):
            
            Año_inicial = np.nan
            Mes_inicial = np.nan
            
            if j < len(B['Año'].unique()+1):
            
                if B['Año'].unique()[j] == 2008:

                    Año_inicial = 2006
                    Mes_inicial = 'enero'

                elif B['Año'].unique()[j] > 2008:

                    Año_inicial = B['Año'].unique()[j-1]
                    Mes_inicial = Paro['Mes'].unique()[int(B[B['Año'] == B['Año'].unique()[j-1]]['Mes'].unique())-1]

                Año_final = B['Año'].unique()[j]
                
                if B['Año'].unique()[j] != 2019:
                    Mes_final = Paro['Mes'].unique()[int(B[B['Año'] == B['Año'].unique()[j]]['Mes'].unique())-1]
                else:
                    Mes_final = Paro['Mes'].unique()[int(B[B['Año'] == B['Año'].unique()[j]]['Mes'].unique()[0])-1]
                
            elif j == len(B['Año'].unique()+1):
                
                if B['Tipo de Elección'].unique()[0] == 'General':
                
                    Año_inicial = B['Año'].unique()[j-2]
                    Mes_inicial = Paro['Mes'].unique()[int(B[B['Año'] == B['Año'].unique()[j-2]]['Mes'].unique())-1]
                
                    Año_final = B['Año'].unique()[j-1]
                    Mes_final = Paro['Mes'].unique()[int(B[B['Año'] == B['Año'].unique()[j-1]]['Mes'].unique()[1])-1]
                    
                else:
                    pass
            
            if math.isnan(Año_inicial) == False:

                try:

                    id1 = C[(C['Año'] == Año_inicial)&(C['Mes'] == Mes_inicial)].index[0]
                    id2 = C[(C['Año'] == Año_final)&(C['Mes'] == Mes_final)].index[0]

                    for k in c:

                        P = C[k][id1:id2].tolist()
                        if C['Año'][id1:id2][-1:].unique()[0] != 2016:
                            chunks = [P[x:x+int(round((len(P)+4)/4,0))] for x in range(0,len(P), int(round((len(P)+4)/4,0)))]
                        else:
                            chunks = [P[x:x+int(round((len(P)+4)/4,0))] for x in [2,2,1,1]]
                        Etapas = ['I','II','III','IV']
                        for n in range(len(Etapas)):

                            id_data = B[(B['Año'] == Año_final)&(B['Mes'] == C['Mes'].unique().tolist().index(Mes_final)+1)].index[0]
                            cont = []

                            for m in range(1,len(chunks[n])):

                                try:
                                    cont.append((chunks[n][m]-chunks[n][m-1])/chunks[n][m-1]*100)
                                except:
                                    pass

                            data.at[id_data, 'Incremento '+k+' '+Etapas[n]] = np.mean(cont)
                
                except:
                    pass
                
    return data

In [51]:
%%time

for i in c:
    for j in ['I','II','III','IV']:
        data['Incremento '+i+' '+j] = np.nan
        
# Incremento en elecciones generales
        
data = Incrementos(data,Paro,c)

# Incremento en las elecciones autonómicas

data = Incrementos(data,Paro,c,tipo = 'Autonómica')

Wall time: 5min 28s


In [52]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Incremento Industria III,Incremento Industria IV,Incremento Construcción I,Incremento Construcción II,Incremento Construcción III,Incremento Construcción IV,Incremento Servicios I,Incremento Servicios II,Incremento Servicios III,Incremento Servicios IV
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1977,-58.74,6.0,Albox,10407.0,Almería,79.30,20.56,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,33.03,11.0,Villanueva del Río y Minas,4860.0,Sevilla,31.83,64.86,General,3.0,...,2.952844,-3.127308,-1.423283,-0.565427,-0.870667,8.052143,-0.581356,0.257084,0.572544,0.251740
18185,2019,19.71,4.0,Villaverde del Río,7797.0,Sevilla,39.64,59.35,General,3.0,...,-2.122091,7.643038,0.515030,-2.232204,0.961459,-6.263434,-1.333473,1.376391,0.325222,1.031891
18186,2019,18.04,11.0,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General,3.0,...,1.376585,-1.204393,1.072223,-0.101085,0.129873,4.121672,-1.264850,1.546503,0.313417,-0.506482
18187,2019,-5.37,4.0,Écija,39882.0,Sevilla,52.02,46.65,General,4.0,...,-1.113220,-3.067928,-1.491405,-1.934562,-0.458961,-5.109077,-0.550460,0.016662,-0.210380,1.481948


### 2.9. Otros datos <a class="anchor" id="9"></a>

#### 2.9.1. Distinción entre municipios costeros y de interior <a class="anchor" id="9.1"></a>

Es un lugar común que en las zonas de costa hay mayor tendencia a votar a la derecha que en el interior. Para comprobar como de cierto es, será necesario distinguirlos.

In [53]:
d = pd.read_csv('Data/municipios_costeros.txt')

In [54]:
data['Costa'] = False
for i in data['Municipios'].unique():
    if i in d['Municipios'].tolist():
        
        data['Costa'][data['Municipios'] == i] = True

In [55]:
data[(data['Costa'] == True)]

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Incremento Industria IV,Incremento Construcción I,Incremento Construcción II,Incremento Construcción III,Incremento Construcción IV,Incremento Servicios I,Incremento Servicios II,Incremento Servicios III,Incremento Servicios IV,Costa
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
12,1977,3.04,6.0,Almería,120072.0,Almería,48.36,51.40,General,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
30,1977,-45.17,6.0,Carboneras,3183.0,Almería,72.55,27.38,General,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
34,1977,-19.39,6.0,Cuevas del Almanzora,7777.0,Almería,59.62,40.23,General,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
37,1977,-54.54,6.0,Enix,509.0,Almería,77.27,22.73,General,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17954,2019,-13.13,11.0,Torremolinos,68262.0,Málaga,55.99,42.86,General,4.0,...,-0.062876,-1.126024,-0.056637,-0.258923,-0.475953,0.712442,1.502147,1.464849,-1.219302,True
17955,2019,-6.07,4.0,Torrox,16465.0,Málaga,52.57,46.50,General,4.0,...,-8.996048,-2.225786,-0.399253,-0.137118,-7.004387,1.733740,0.957854,-0.590970,-3.686157,True
17956,2019,-10.56,11.0,Torrox,16465.0,Málaga,54.71,44.15,General,4.0,...,-1.342971,-3.487583,0.003071,0.470558,0.062630,0.718167,0.394884,0.402728,0.233351,True
17971,2019,-9.16,4.0,Vélez-Málaga,80817.0,Málaga,54.08,44.92,General,4.0,...,-3.779541,-1.523199,-2.095559,-1.062944,-1.235266,-0.090863,0.583845,-0.529798,-0.250037,True


#### 2.9.2. Gobiernos Nacionales y Autonómicos <a class="anchor" id="9.2"></a>

En este apartado se crearán columnas con el signo de los gobiernos nacional y autonómico de la legislatura anterior a cada elección.

In [56]:
data['Gobierno Nacional'] = 'Derecha'
data['Gobierno Autonómico'] = 'Izquierda'

Para el Gobierno Nacional:

In [57]:
# Se definen los años de Gobierno izquierdista

Año = [1986,1989,1990,1993,1996,2008,2011,2019]

for i in Año:
    data.loc[data['Año'] == i, 'Gobierno Nacional'] = 'Izquierda'

Para el Gobierno Autonómico:

In [58]:
data.loc[data['Año'] < 1982, 'Gobierno Autonómico'] = np.nan
data.loc[data['Año'] == 2019, 'Gobierno Autonómico'] = 'Derecha'

In [59]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Incremento Construcción II,Incremento Construcción III,Incremento Construcción IV,Incremento Servicios I,Incremento Servicios II,Incremento Servicios III,Incremento Servicios IV,Costa,Gobierno Nacional,Gobierno Autonómico
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Derecha,NaN
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Derecha,NaN
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Derecha,NaN
3,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Derecha,NaN
4,1977,-58.74,6.0,Albox,10407.0,Almería,79.30,20.56,General,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Derecha,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,33.03,11.0,Villanueva del Río y Minas,4860.0,Sevilla,31.83,64.86,General,3.0,...,-0.565427,-0.870667,8.052143,-0.581356,0.257084,0.572544,0.251740,False,Izquierda,Derecha
18185,2019,19.71,4.0,Villaverde del Río,7797.0,Sevilla,39.64,59.35,General,3.0,...,-2.232204,0.961459,-6.263434,-1.333473,1.376391,0.325222,1.031891,False,Izquierda,Derecha
18186,2019,18.04,11.0,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General,3.0,...,-0.101085,0.129873,4.121672,-1.264850,1.546503,0.313417,-0.506482,False,Izquierda,Derecha
18187,2019,-5.37,4.0,Écija,39882.0,Sevilla,52.02,46.65,General,4.0,...,-1.934562,-0.458961,-5.109077,-0.550460,0.016662,-0.210380,1.481948,False,Izquierda,Derecha


#### 2.9.3. Número de partidos importantes por bloque <a class="anchor" id="9.3"></a>

Puede suponerse que el volumen total de un bloque se ve afectado por la cantidad de oferta de ese bloque, principalmente si se percibe como útil. De modo que será razonable considerar dos columnas que indiquen el número de partidos importantes que se presentan por bloque a las elecciones.

Es díficil elaborar una definición precisa del significado del término *partido importante*, sobre todo a priori. La pauta principal podría ser si las encuestas, de manera general, otorgan representación a un partido, así como su exposición mediática. De esta forma, pueden siderarse partidos grandes:

* De izquierdas

    * Partido Socialista Obrero Español de Andalucía (PSOE-A)
    * Izquierda Unida en todas sus fórmulas (PCA/IU-CA/IUCA-LV)
    * Partido Andalucista (PA), hasta 2012
    * Adelante Andalucía (AA)
    
* De derechas

    * Unión de Centro Democrático (UCD)
    * Alianza Popular/Partido Popular (AP/PP)
    * Ciudadanos - Partido de la Ciudadanía (C's)
    * Vox (VOX), a partir de 2018

In [60]:
Aut = [[3,2],[3,1],[3,1],[3,1],[3,1],[3,1],[2,1],[2,1],[3,2],[2,3]]
Gen = [[3,2],[3,2],[2,2],[2,2],[3,1],[2,1],[2,1],[3,1],[3,1],[2,1],[3,1],[3,2],[2,2],[2,3]]

for i in data['Tipo de Elección'].unique():
    A = data[data['Tipo de Elección'] == i]
    
    if i == 'General':
        for j in range(len(A['Año'].unique())):
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Partidos Izquierda'] = Gen[j][0]
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Partidos Derecha'] = Gen[j][1]
            
    if i == 'Autonómica':
        for j in range(len(A['Año'].unique())):
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Partidos Izquierda'] = Aut[j][0]
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Partidos Derecha'] = Aut[j][1]

In [61]:
data[data['Tipo de Elección'] == 'Autonómica']

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Incremento Construcción IV,Incremento Servicios I,Incremento Servicios II,Incremento Servicios III,Incremento Servicios IV,Costa,Gobierno Nacional,Gobierno Autonómico,Partidos Izquierda,Partidos Derecha
1518,1982,-33.09,5.0,Abla,1723.0,Almería,66.54,33.45,Autonómica,2.0,...,NaN,NaN,NaN,NaN,NaN,False,Derecha,Izquierda,3.0,2.0
1520,1982,27.01,5.0,Abrucena,1649.0,Almería,36.49,63.50,Autonómica,2.0,...,NaN,NaN,NaN,NaN,NaN,False,Derecha,Izquierda,3.0,2.0
1522,1982,16.16,5.0,Adra,17415.0,Almería,41.92,58.08,Autonómica,4.0,...,NaN,NaN,NaN,NaN,NaN,True,Derecha,Izquierda,3.0,2.0
1524,1982,-25.78,5.0,Alboloduy,1060.0,Almería,62.89,37.11,Autonómica,2.0,...,NaN,NaN,NaN,NaN,NaN,False,Derecha,Izquierda,3.0,2.0
1526,1982,-15.99,5.0,Albox,10113.0,Almería,58.00,42.01,Autonómica,4.0,...,NaN,NaN,NaN,NaN,NaN,False,Derecha,Izquierda,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16644,2018,33.22,12.0,Villanueva de San Juan,1155.0,Sevilla,33.38,66.60,Autonómica,2.0,...,11.516544,1.499856,0.901140,2.624244,1.412347,False,Derecha,Izquierda,2.0,3.0
16645,2018,-1.50,12.0,Villanueva del Ariscal,6571.0,Sevilla,50.62,49.12,Autonómica,3.0,...,-1.700213,-0.268044,-0.745486,0.014332,-1.093570,False,Derecha,Izquierda,2.0,3.0
16646,2018,29.49,12.0,Villanueva del Río y Minas,4860.0,Sevilla,35.12,64.61,Autonómica,3.0,...,0.133999,-1.026241,-0.577170,-0.265250,0.285380,False,Derecha,Izquierda,2.0,3.0
16647,2018,12.45,12.0,Villaverde del Río,7797.0,Sevilla,43.75,56.20,Autonómica,3.0,...,0.698988,-0.171153,-0.662555,0.934000,0.775101,False,Derecha,Izquierda,2.0,3.0


#### 2.9.4. Eventualidades <a class="anchor" id="9.4"></a>

Es obvio que existen una serie de hechos puntuales que pueden influir en poco tiempo y de forma decisiva en los resultados electorales. El caso paradigmático en España son los atentados del 11 de marzo, ocurridos días antes de las elecciones, y todo lo que se derivó de ellos. También el estallido de casos de corrupción y su difusión mediática, aunque sea menos puntual y más contínuo. Se va a resumir, pues, en una columna, todo lo relacionado con estos casos, a saber:

* Eventos políticos:
    * Transición: 1977-1979
    * Golpe de Estado del 23 de Febrero: 1982
    * Moción de Censura: 2018-abril de 2019
    * Repetición de Elecciones: 2016, novimbre de 2019
* Corrupción: 1993, 1994, 1996, 2015
* Atentado: 2004

In [62]:
data['Eventualidades'] = np.nan

data['Eventualidades'][data['Año'] == 1977] = 'Transición'
data['Eventualidades'][data['Año'] == 1979] = 'Transición'
data['Eventualidades'][data['Año'] == 1982] = 'Golpe de Estado'
data['Eventualidades'][data['Año'] == 1993] = 'Corrupción'
data['Eventualidades'][data['Año'] == 1994] = 'Corrupción'
data['Eventualidades'][data['Año'] == 1996] = 'Corrupción'
data['Eventualidades'][data['Año'] == 2015] = 'Corrupción'
data['Eventualidades'][data['Año'] == 2018] = 'Corrupción'
data['Eventualidades'][data['Año'] == 2016] = 'Repetición'
data['Eventualidades'][(data['Año'] == 2019)&(data['Mes'] == 11)] = 'Repetición'
data['Eventualidades'][(data['Año'] == 2019)&(data['Mes'] == 4)] = 'Censura'
data['Eventualidades'][data['Año'] == 2004] = 'Atentado'

In [63]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Incremento Servicios I,Incremento Servicios II,Incremento Servicios III,Incremento Servicios IV,Costa,Gobierno Nacional,Gobierno Autonómico,Partidos Izquierda,Partidos Derecha,Eventualidades
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,...,NaN,NaN,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,...,NaN,NaN,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,...,NaN,NaN,NaN,NaN,True,Derecha,NaN,3.0,2.0,Transición
3,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,...,NaN,NaN,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición
4,1977,-58.74,6.0,Albox,10407.0,Almería,79.30,20.56,General,4.0,...,NaN,NaN,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,33.03,11.0,Villanueva del Río y Minas,4860.0,Sevilla,31.83,64.86,General,3.0,...,-0.581356,0.257084,0.572544,0.251740,False,Izquierda,Derecha,2.0,3.0,Repetición
18185,2019,19.71,4.0,Villaverde del Río,7797.0,Sevilla,39.64,59.35,General,3.0,...,-1.333473,1.376391,0.325222,1.031891,False,Izquierda,Derecha,2.0,3.0,Censura
18186,2019,18.04,11.0,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General,3.0,...,-1.264850,1.546503,0.313417,-0.506482,False,Izquierda,Derecha,2.0,3.0,Repetición
18187,2019,-5.37,4.0,Écija,39882.0,Sevilla,52.02,46.65,General,4.0,...,-0.550460,0.016662,-0.210380,1.481948,False,Izquierda,Derecha,2.0,3.0,Censura


#### 2.9.5. Meteorología <a class="anchor" id="9.5"></a>

Aunque pueda parecer un dato peregrino, el tiempo el día de la elección tiene una cierta influencia en la participación electoral. Por supuesto, esto no tiene que estar relacionado, necesariamente, con mayor abstención en el electorado de cualquiera de los bloques, pero vale la pena tenerlo en cuenta. Así, se incluirán dos columnas, una indicando si hubo o no precipitaciones, y otra con la temperatura media. Los datos meteorológicos se tomarán de la página web de la [AEMET](https://datosclima.es/Aemethistorico/Meteosingleday.php).

In [64]:
Aut = [['No',26.0],['No',19.6],['No',27.4],['No',23.8],['No',11.7],['No',11.4],['No',12.6],['No',26.9],['Débil',26.9],
       ['No',10.9]]
Gen = [['No',21.6],['No',8.7],['No',18.9],['No',17.4],['Débil',20],['No',11.7],['No',15.7],['No',11.4],['No',12.6],
       ['Moderada',13.1],['No',26.9],['No',24.5],['No',19.0],['No',13.3]]

for i in data['Tipo de Elección'].unique():
    A = data[data['Tipo de Elección'] == i]
    
    if i == 'General':
        for j in range(len(A['Año'].unique())):
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Precipitaciones'] = Gen[j][0]
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Temperatura'] = Gen[j][1]
            
    if i == 'Autonómica':
        for j in range(len(A['Año'].unique())):
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Precipitaciones'] = Aut[j][0]
            data.loc[(data['Tipo de Elección'] == i)&(data['Año'] == A['Año'].unique()[j]), 'Temperatura'] = Aut[j][1]

In [65]:
data

,Año,Diferencia,Mes,Municipios,Población,Provincia,Resultado Derecha,Resultado Izquierda,Tipo de Elección,Clasificación,...,Incremento Servicios III,Incremento Servicios IV,Costa,Gobierno Nacional,Gobierno Autonómico,Partidos Izquierda,Partidos Derecha,Eventualidades,Precipitaciones,Temperatura
0,1977,-80.40,6.0,Abla,1755.0,Almería,90.20,9.80,General,2.0,...,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición,No,21.6
1,1977,-52.33,6.0,Abrucena,1828.0,Almería,76.11,23.78,General,2.0,...,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición,No,21.6
2,1977,-20.68,6.0,Adra,15602.0,Almería,60.26,39.58,General,4.0,...,NaN,NaN,True,Derecha,NaN,3.0,2.0,Transición,No,21.6
3,1977,-68.04,6.0,Alboloduy,1082.0,Almería,84.02,15.98,General,2.0,...,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición,No,21.6
4,1977,-58.74,6.0,Albox,10407.0,Almería,79.30,20.56,General,4.0,...,NaN,NaN,False,Derecha,NaN,3.0,2.0,Transición,No,21.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18184,2019,33.03,11.0,Villanueva del Río y Minas,4860.0,Sevilla,31.83,64.86,General,3.0,...,0.572544,0.251740,False,Izquierda,Derecha,2.0,3.0,Repetición,No,13.3
18185,2019,19.71,4.0,Villaverde del Río,7797.0,Sevilla,39.64,59.35,General,3.0,...,0.325222,1.031891,False,Izquierda,Derecha,2.0,3.0,Censura,No,13.3
18186,2019,18.04,11.0,Villaverde del Río,7797.0,Sevilla,40.26,58.30,General,3.0,...,0.313417,-0.506482,False,Izquierda,Derecha,2.0,3.0,Repetición,No,13.3
18187,2019,-5.37,4.0,Écija,39882.0,Sevilla,52.02,46.65,General,4.0,...,-0.210380,1.481948,False,Izquierda,Derecha,2.0,3.0,Censura,No,13.3


Finalmente, puede guardarse el dataset final.

In [66]:
data.to_csv('Resultados/Data.csv', index = False)